In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pyperclip
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from pprint import pprint
from tqdm import tqdm, notebook
import json
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import requests
import shutil

In [3]:
path = r'C:\Users\재욱\Downloads\chromedriver.exe'
driver = webdriver.Chrome(path)
driver.implicitly_wait(2)

companies_url = "https://www.rocketpunch.com/companies" 
driver.get(companies_url)

xpath = '//*[@id="company-list"]'
division = driver.find_element_by_xpath(xpath)
company_img_list = division.find_elements_by_tag_name('img')
company_name_list = division.find_elements_by_tag_name('strong')

# country_url_list = []
# for country_url in country_list:
#     country_url_list.append(country_url.get_attribute('src'))

print(company_img_list)
print(company_name_list)

//*[@id="company-list"]/div[1]/div[2]/a/h4/strong
//*[@id="company-list"]/div[1]/div[1]/div/img

[<selenium.webdriver.remote.webelement.WebElement (session="6251e6cbf4e69077c4cc73ff7d2ebed4", element="ed358e00-7de4-4ec2-b02f-2a91e2731e5a")>, <selenium.webdriver.remote.webelement.WebElement (session="6251e6cbf4e69077c4cc73ff7d2ebed4", element="108d3270-4eb2-4f40-a938-7ef60aa6d7e2")>, <selenium.webdriver.remote.webelement.WebElement (session="6251e6cbf4e69077c4cc73ff7d2ebed4", element="d5c1c7ff-febc-41a3-b751-0262ce8b5361")>, <selenium.webdriver.remote.webelement.WebElement (session="6251e6cbf4e69077c4cc73ff7d2ebed4", element="d76826ed-35a3-426c-8204-ec2cf04b88dc")>, <selenium.webdriver.remote.webelement.WebElement (session="6251e6cbf4e69077c4cc73ff7d2ebed4", element="09c10955-6cb3-4be3-bbf2-63f316b1e12b")>, <selenium.webdriver.remote.webelement.WebElement (session="6251e6cbf4e69077c4cc73ff7d2ebed4", element="5a49037f-ac03-4e25-b15d-8944441fddf5")>, <selenium.webdriver.remote.webelement.WebElement (session="6251e6cbf4e69077c4cc73ff7d2ebed4", element="6527f11a-e14f-45f3-83ef-94efb6aa

In [ ]:
def save_images(file_path, image_url_list, name_list):
    
    img_folder_path = f'./brands_of_the_world/{country_name}_company_logo/'    
    os.makedirs(img_folder_path, exist_ok=True)
    
    for i, image_url in enumerate(image_url_list):
        try:
            response = requests.get(image_url.get_attribute('src'), stream=True)
            with open(img_folder_path + name_list[i].text +'.jpg','wb') as out_file:
                shutil.copyfileobj(response.raw, out_file)
                    
        except Exception as e:
            print(e)

In [ ]:
def read_json(file_path):
    with open(file_path, "r") as fp:
        data = json.loads(fp.read())
    return data

def write_json(data, file_path):
    with open(file_path, "w") as fp:
        json.dump(data, fp)
        
# 보통 file pointer -> fp 사용

In [ ]:
# 빈 딕셔너리 (dict_status) 선언 후 JSON 파일 먼저 만들기
try:
    XPATH_COUNTRY_TABLE = '//*[@id="primaryInner"]/div/div[1]'
    XPATH_COUNTRY_NAME = '//*[@id="primary"]/h1/span'
    print("지난 status 상황 : ")
    dict_status = read_json("brands_of_the_world.json")
    pprint(dict_status)
    print('+-----------------------------------------------------------------+')

    # COUNTRY_NUMBER 대신 enumerate를 쓰는 연습
    for idx , country_path in enumerate(notebook.tqdm(country_url_list, desc='iterate range 100')):
        driver.get(country_path)
        country_name = driver.find_element_by_xpath(XPATH_COUNTRY_NAME).text
        
        # 해당 국가의 key-value가 없을 경우. (해당 국가 페이지 처음 방문)
        if country_name not in dict_status.keys():
            dict_status[country_name] = {"status" : 'NOT DONE', "latest_page" : -1} # 현재 국가 save status 초기화
            page_num = 0
        # 해당 국가의 key-value가 있을 경우. DONE이든 NOT DONE이든 (해당 국가 페이지 방문한 적 있음)
        else: 
            page_num = dict_status[country_name]["latest_page"]
            
        # 해당 국가의 value의 status key가 'NOT DONE'일 경우(해당 국가 페이지 쭉 방문해야함.)    
        if dict_status[country_name]["status"] != 'DONE':   
            
            while True:
                driver.get(f'{country_path}?page={page_num}') #f string 사용!
                division = driver.find_element_by_xpath(XPATH_COUNTRY_TABLE)
                name_list = division.find_elements_by_tag_name('li') # List형
                image_url_list = division.find_elements_by_tag_name('img') # 이미지 URL
                
                if len(image_url_list) == 0:
                    break
                    
                save_images(country_name, image_url_list, name_list)
                
                print(f"{country_name}의 {page_num}번째 page 이미지 저장 완료")
                dict_status[country_name]["latest_page"] +=1
                page_num +=1
                    
            dict_status[country_name]["status"] = 'DONE'
            write_json(dict_status, "brands_of_the_world.json")
            pprint(dict_status)
            print(f'{idx}번째 나라 {country_name}의 모든 이미지 저장 완료')
            print('+-----------------------------------------------------------------+')
         
except Exception as e:
    print(e)
except KeyboardInterrupt:
    print("[KeyboardInterrupt] 현재까지의 status :")
    with open("brands_of_the_world.json", "w") as upload_file:
        json.dump(dict_status, upload_file)
    pprint(dict_status)
    print("\n")
except:
    print('Other error')